<a href="https://colab.research.google.com/github/ajakrish/Ajay_info5502_spring2022/blob/main/lab_assignment_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The seventh Lab-assignment (03/31/2022, 50 points in total)

The purpose of this exercise is to build several regression models based on real-world datasets. 

### Question 1 (25 points) Build a regession model to predict the change in price of the stock overtime. It can either be linear regression or non-linear regression. You can download the dataset from here: https://github.com/unt-iialab/info5502-spring2022/blob/main/datasets/portfolio_data.csv. The implementation of the LSTM on this same dataset was posted here as an reference: https://www.kaggle.com/code/faressayah/stock-market-analysis-prediction-using-lstm/notebook

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [15]:
import pandas as pd

# load the dataset
data = pd.read_csv("/content/sample_data/portfolio_data.csv")
data.head(5)

,Date,AMZN,DPZ,BTC,NFLX
0,5/1/2013,248.229996,51.190983,106.250000,30.415714
1,5/2/2013,252.550003,51.987320,98.099998,30.641428
2,5/3/2013,258.049988,52.446388,112.900002,30.492857
3,5/6/2013,255.720001,53.205257,109.599998,30.098572
4,5/7/2013,257.730011,54.151505,113.199997,29.464285


In [16]:
# dropping Date and NFLX xolumns
x = data.drop(['Date', 'NFLX'], axis = 1)

y = data['NFLX']


In [17]:
#split the  data in train and test
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 0)

lm = LinearRegression()
lm.fit(xtrain, ytrain)

ypred = lm.predict(xtest)

print("RMSE score is",np.sqrt(mean_squared_error(ytest, ypred)))
print("R2 Score is", r2_score(ytest, ypred))

RMSE score is 21.479712235055025
R2 Score is 0.9597280743181571


In [18]:
from sklearn.tree import DecisionTreeRegressor

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 0)

dt_reg = DecisionTreeRegressor()
dt_reg.fit(xtrain, ytrain)

ypred = dt_reg.predict(xtest)

print("RMSE score is",np.sqrt(mean_squared_error(ytest, ypred)))
print("R2 Score is", r2_score(ytest, ypred))

RMSE score is 14.56098634651086
R2 Score is 0.9814933816686771


In [19]:
from sklearn.ensemble import RandomForestRegressor

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 0)

rfreg = RandomForestRegressor(random_state=211)
rfreg.fit(xtrain, ytrain)

rfregpred = rfreg.predict(xtest)
print("RMSE score is",np.sqrt(mean_squared_error(ytest, rfregpred)))
print("R2 Score is", r2_score(ytest, rfregpred))

RMSE score is 11.106742773748609
R2 Score is 0.9892324001668802


### Question 2 (25 points) The objective of this task is to detect hate speech in tweets. For the sake of simplicity, we say a tweet contains hate speech if it has a racist or sexist sentiment associated with it. So, the task is to classify racist or sexist tweets from other tweets. Formally, given a training sample of tweets and labels, where label '1' denotes the tweet is racist/sexist and label '0' denotes the tweet is not racist/sexist, your objective is to predict the labels on the test dataset. The dataset can be downloaded here: https://github.com/unt-iialab/info5502-spring2022/tree/main/datasets/Twitter%20Sentiment%20Analysis. Please build a logistic regression model for the classification, evaluation metrics include accuracy, precision, recall, and F-1 score need to be reported. 

In [20]:
### You code here:
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle
import nltk
nltk.download('stopwords')








[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
data = pd.read_csv("/content/sample_data/train.csv")


In [22]:
# getting stopwords of english
stopwords = stopwords.words("english")
stopwords = [re.sub('[^a-zA-Z0-9]+', '', word) for word in stopwords]

# removing all special characters
data['cleaned_tweets'] = data['tweet'].str.replace('[^a-zA-Z0-9 ]', '')
data['cleaned_tweets'] = data['cleaned_tweets'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
# applying lemmatization on the tweets
data['cleaned_tweets'] = data['cleaned_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="v") for word in row.split()))
data['cleaned_tweets'] = data['cleaned_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="n") for word in row.split()))
data['cleaned_tweets'] = data['cleaned_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="a") for word in row.split()))
data['cleaned_tweets'] = data['cleaned_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="r") for word in row.split()))
from sklearn.linear_model import LogisticRegression
X = data['cleaned_tweets']
Y = data['label']

In [24]:

vectorizer = TfidfVectorizer(min_df = 3, ngram_range=(1, 2))

vectorizer = vectorizer.fit(X)

X_features = vectorizer.transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X_features, Y, test_size=0.2)

lr = LogisticRegression()
model = lr.fit(X_train, y_train)

print("Classification Report")
print(classification_report(y_test, model.predict(X_test)))

print("Confusion Matrix:")
print(confusion_matrix(y_test, model.predict(X_test)))

Classification Report
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5923
           1       0.90      0.30      0.45       470

    accuracy                           0.95      6393
   macro avg       0.93      0.65      0.71      6393
weighted avg       0.94      0.95      0.93      6393

Confusion Matrix:
[[5908   15]
 [ 328  142]]


In [25]:
print(X_features.shape)
(31962, 17047)
test_data = pd.read_csv("/content/sample_data/test.csv")

test_data['cleaned_tweets'] = test_data['tweet'].str.replace('[^a-zA-Z0-9 ]', '')
test_data['cleaned_tweets'] = test_data['cleaned_tweets'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

test_data['cleaned_tweets'] = test_data['cleaned_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="v") for word in row.split()))
test_data['cleaned_tweets'] = test_data['cleaned_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="n") for word in row.split()))
test_data['cleaned_tweets'] = test_data['cleaned_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="a") for word in row.split()))
test_data['cleaned_tweets'] = test_data['cleaned_tweets'].apply(lambda row: " ".join(wordnet_lemmatizer.lemmatize(word, pos="r") for word in row.split()))

test_data_features = vectorizer.transform(test_data['cleaned_tweets'])#.toarray()
print(test_data_features.shape)



test_data_pred = model.predict(test_data_features)
print(test_data_pred)

(31962, 17047)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


(17197, 17047)
[0 0 0 ... 0 0 0]
